# Currency risks for Swiss residents


## CMD: A Short Summary

If one wants to easily replicate the main findings of this paper, they might execute the command lines below (from the root folder). Later, we'll explore each part in further detail and explore the more nuanced analysis allowed by our implementations.

In [1]:
:: Setup
setup.bat

:: Data retrieval and processing
python src/data/fred_data_retrieval.py
python src/data/oecd_data_retrieval.py
python src/data/fred_oecd_data_concatenation.py

:: Analysis, modeling and reporting
Rscript src/models/CODE_FAMA_CURRENCY.R
python src/analysis/performances.py

:: LaTeX files compiling

SyntaxError: invalid syntax (550391262.py, line 1)

## Data retrieval

We use FRED to gather ForEx and VIX monthly data through FRED's API. We additionally gather German and Italian 10y-bond yields.

We obtain short-term interest rates through an html request to the OECD database.

CAUTION: OECD data for Japan (JPY) is available only since April 2002. Thus, it's not recommended to use a start date previous to this one.

### FRED

In [2]:
import pandas as pd
import src.data.fred_data_retrieval as fred

# Define a time horizon
start_date = pd.Timestamp('2002-04-01')
end_date = pd.Timestamp('2023-10-31')

# Enter FRED API key
api_key = '41641234dac871704cd09b2b8dd163ac'

# Call to download function
fred.download(start_date=start_date,
              end_date=end_date,
              api_key=api_key)


### OECD

In [3]:
import src.data.oecd_data_retrieval as oecd

oecd.download(start_date=start_date,
              end_date=end_date)

### Merge

In [4]:
import src.data.fred_oecd_data_concatenation as merge
merge.merge()

### Load data

Now, we'll open the downloaded monthly data at a pandas Data Frame:

In [5]:
df = pd.read_csv('data/processed/monthly_data.csv')

# Standardize data handling with pandas Timestamp
df['Date'] = df['Date'].map(lambda x: pd.Timestamp(x))
df.set_index('Date', inplace=True)

# Visualize the requested data
df.head(6)[['USD', 'GBP', 'EUR', 'VIX', 'USDi', 'GBPi', 'EURi', 'IT_minus_DE']]

,USD,GBP,EUR,VIX,USDi,GBPi,EURi,IT_minus_DE
Date,,,,,,,,
2002-04-30,1.6216,2.361860,1.459764,21.91,1.87,4.17319,3.4069,0.248762
2002-05-31,1.5693,2.295101,1.465569,19.98,1.82,4.15370,3.4671,0.238273
2002-06-30,1.4920,2.274554,1.470515,25.40,1.81,4.17758,3.4640,0.238273
2002-07-31,1.4833,2.317656,1.453041,32.03,1.79,4.06054,3.4100,0.241609
2002-08-31,1.5007,2.322483,1.471586,32.64,1.73,3.98654,3.3519,0.238455
2002-09-30,1.4758,2.317006,1.457943,39.69,1.76,3.98872,3.3101,0.238455


## Analysis

### Regression results
We use R to compute the (log) forward rates and returns, and subsequently perform the linear regression in analysis.

The following command line reproduces the regression and publishes the results onto the LaTeX report, accessible at latex/paper.

### Installing IRkernel on the JupyterNotebook


First need to install and load the interactive kernel, in order to run code. In order to use the R code into the jupiterNotebook use the commande %%R.

In [6]:
!pip install rpy2 -q


In [7]:
%load_ext rpy2.ipython

- Project '~/Documents/GitHub/currency_risk_ch' loaded. [renv 1.0.3]
- The project is out-of-sync -- use `renv::status()` for details.


In [8]:
%%R
source("src/models/CODE_FAMA_CURRENCY.R")

The following package(s) will be installed:
- renv [1.0.3]
These packages will be installed into "~/Library/Caches/org.R-project.R/R/renv/library/currency_risk_ch-1d6cd68c/R-4.1/x86_64-apple-darwin17.0".

Do you want to proceed? [Y/n]: y

# Installing packages --------------------------------------------------------
- Installing renv ...                           OK [linked from cache]
- The library is already synchronized with the lockfile.
Rows: 259 Columns: 22
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl  (21): USD, EUR, JPY, GBP, AUD, CAD, NZD, SEK, NOK, VIX, IT_minus_DE, US...
date  (1): Date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.

% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Wed, Dec 13, 2023 - 09:11:09
% Requires LaTeX packages

\hline \\[-1.8ex] 
Observations & \multicolumn{1}{c}{258} & \multicolumn{1}{c}{257} & \multicolumn{1}{c}{258} \\ 
R$^{2}$ & \multicolumn{1}{c}{0.496} & \multicolumn{1}{c}{0.392} & \multicolumn{1}{c}{0.215} \\ 
Adjusted R$^{2}$ & \multicolumn{1}{c}{0.490} & \multicolumn{1}{c}{0.385} & \multicolumn{1}{c}{0.205} \\ 
Residual Std. Error & \multicolumn{1}{c}{0.013 (df = 254)} & \multicolumn{1}{c}{0.022 (df = 253)} & \multicolumn{1}{c}{0.026 (df = 254)} \\ 
F Statistic & \multicolumn{1}{c}{83.310$^{***}$ (df = 3; 254)} & \multicolumn{1}{c}{54.480$^{***}$ (df = 3; 253)} & \multicolumn{1}{c}{23.147$^{***}$ (df = 3; 254)} \\ 
\hline 
\hline \\[-1.8ex] 
\textit{Note:}  & \multicolumn{3}{r}{$^{*}$p$<$0.1; $^{**}$p$<$0.05; $^{***}$p$<$0.01} \\ 
\end{tabular} 
\end{table} 

% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Wed, Dec 13, 2023 - 09:11:18
% Requires LaTeX packages: dcolumn 
\begin{table}[!htbp] \centering 


F Statistic         62.239*** (df = 4; 253) 42.678*** (df = 4; 252) 17.406*** (df = 4; 253)
Note:                                                           *p<0.1; **p<0.05; ***p<0.01

                                    Dependent variable:      
                               ------------------------------
                                  GBP       AUD       CAD    
                                  (1)       (2)       (3)    
-------------------------------------------------------------
F_S_GBP                          0.212                       
                                (1.489)                      
                                                             
F_S_AUD                                    0.821             
                                          (1.210)            
                                                             
F_S_CAD                                              0.046   
                                                    (1.706)  
         


Attachement du package : ‘dplyr’

Les objets suivants sont masqués depuis ‘package:stats’:

    filter, lag

Les objets suivants sont masqués depuis ‘package:base’:

    intersect, setdiff, setequal, union

Le chargement a nécessité le package : zoo

Attachement du package : ‘zoo’

Les objets suivants sont masqués depuis ‘package:base’:

    as.Date, as.Date.numeric


Please cite as: 

 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.
 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 



In [9]:
%%R
str(monthly_data)

spc_tbl_ [259 × 78] (S3: spec_tbl_df/tbl_df/tbl/data.frame)
 $ Date           : Date[1:259], format: "2002-04-30" "2002-05-31" ...
 $ USD            : num [1:259] 1.62 1.57 1.49 1.48 1.5 ...
 $ EUR            : num [1:259] 1.46 1.47 1.47 1.45 1.47 ...
 $ JPY            : num [1:259] 0.0126 0.0126 0.0124 0.0124 0.0126 ...
 $ GBP            : num [1:259] 2.36 2.3 2.27 2.32 2.32 ...
 $ AUD            : num [1:259] 0.871 0.888 0.84 0.808 0.825 ...
 $ CAD            : num [1:259] 1.034 1.027 0.982 0.936 0.963 ...
 $ NZD            : num [1:259] 0.725 0.752 0.725 0.695 0.702 ...
 $ SEK            : num [1:259] 0.158 0.161 0.162 0.156 0.16 ...
 $ NOK            : num [1:259] 0.193 0.196 0.198 0.195 0.199 ...
 $ VIX            : num [1:259] 21.9 20 25.4 32 32.6 ...
 $ IT_minus_DE    : num [1:259] 0.249 0.238 0.238 0.242 0.238 ...
 $ USDi           : num [1:259] 1.87 1.82 1.81 1.79 1.73 1.76 1.73 1.39 1.34 1.29 ...
 $ NZDi           : num [1:259] 5.68 5.83 5.96 6 5.87 5.86 5.88 5.9 5.92 5.86 ..

With the str() command we can check all the variables, and source() allows use to make sure that we use the right folder. 

### Printing the regression results

The regression contains the coefficients for each parameters and the Durwin-Watson test. 


In [10]:
%%R

print(regression_results)

  Currency      Constant         F_S          VIX        AFX DW_Test
1      EUR -0.0017845508 -1.59796012 -0.001956955 -0.6404679     2.1
2      USD -0.0014400827 -0.72785121  0.040319393 -0.9060323    1.92
3      JPY -0.0019555948 -2.17600085  0.052755479 -0.5573419    2.04
4      GBP  0.0002328140  0.90156517  0.004921689 -0.9621974    2.06
5      AUD  0.0014097004  0.34507082 -0.038318860 -1.0746671    2.08
6      CAD  0.0006460234 -0.06686504 -0.005033763 -1.3035400     2.1
7      NZD  0.0046927036  1.39532647 -0.038052644 -0.8618418    1.99
8      SEK -0.0014002161 -0.31866233 -0.012722785 -0.7949087    2.07
9      NOK -0.0004141796  0.70993097 -0.025185078 -0.9206379    2.13


The table provides an overview for the coefficient without the significance.

In [11]:
%%R


print(dw_tests)


[[1]]

	Durbin-Watson test

data:  Reg_EUR
DW = 2.0993, p-value = 0.7788
alternative hypothesis: true autocorrelation is greater than 0


[[2]]

	Durbin-Watson test

data:  Reg_USD
DW = 1.9215, p-value = 0.2532
alternative hypothesis: true autocorrelation is greater than 0


[[3]]

	Durbin-Watson test

data:  Reg_JPY
DW = 2.0368, p-value = 0.6037
alternative hypothesis: true autocorrelation is greater than 0


[[4]]

	Durbin-Watson test

data:  Reg_GBP
DW = 2.056, p-value = 0.6619
alternative hypothesis: true autocorrelation is greater than 0


[[5]]

	Durbin-Watson test

data:  Reg_AUD
DW = 2.0821, p-value = 0.7356
alternative hypothesis: true autocorrelation is greater than 0


[[6]]

	Durbin-Watson test

data:  Reg_CAD
DW = 2.1008, p-value = 0.7822
alternative hypothesis: true autocorrelation is greater than 0


[[7]]

	Durbin-Watson test

data:  Reg_NZD
DW = 1.9896, p-value = 0.4541
alternative hypothesis: true autocorrelation is greater than 0


[[8]]

	Durbin-Watson test

data:  

The Durwin watson test is great to check autocorrelation. If the results are not significance then there is no strong evidence for auticorrelation

### Robustness test
Including the spread into the regression

In [12]:
%%R
stargazer(models_list_Robust1, align = TRUE, type = "text", out = "latex/tables/regression_table_EUR_USD_JPY_R.tex")



                                              Dependent variable:                          
                    -----------------------------------------------------------------------
                              EUR                     USD                     JPY          
                              (1)                     (2)                     (3)          
-------------------------------------------------------------------------------------------
F_S_EUR                     -1.547                                                         
                            (1.397)                                                        
                                                                                           
F_S_USD                                             -1.291                                 
                                                    (1.416)                                
                                                                               

In [13]:
%%R
stargazer(models_list_Robust2, align = TRUE, type = "text", out = "latex/tables/regression_table_GBP_AUD_CAD_R.tex")



                                    Dependent variable:      
                               ------------------------------
                                  GBP       AUD       CAD    
                                  (1)       (2)       (3)    
-------------------------------------------------------------
F_S_GBP                          0.212                       
                                (1.489)                      
                                                             
F_S_AUD                                    0.821             
                                          (1.210)            
                                                             
F_S_CAD                                              0.046   
                                                    (1.706)  
                                                             
delta_Log_VIX                    0.005   -0.039***   -0.005  
                                (0.006)   (0.006)   (0.005)  
       

In [14]:
%%R
stargazer(models_list_Robust3, align = TRUE, type = "text", out = "latex/tables/regression_table_NZD_SEK_NOK_R.tex")



                                    Dependent variable:     
                               -----------------------------
                                  NZD       SEK       NOK   
                                  (1)       (2)       (3)   
------------------------------------------------------------
F_S_NZD                         2.475*                      
                                (1.465)                     
                                                            
F_S_SEK                                   -0.238            
                                          (1.585)           
                                                            
F_S_NOK                                              0.929  
                                                    (1.497) 
                                                            
delta_Log_VIX                  -0.038*** -0.013**  -0.025***
                                (0.007)   (0.005)   (0.006) 
                       

### FX rates in CHF

The following plot reproduces the FX rates for the specified countries list:

In [12]:
import src.analysis.performances as plot

currencies_list = ['USD', 'GBP', 'JPY']
df2 = plot.build_df(currencies_list)
returns = plot.compute_returns(df2, currencies_list=currencies_list)

In [ ]:
plot.plot_performances(returns)

### Excess returns on Carry FX strategy

We also analyze the profitability and risks associated with investing in the short-term rates in each of the other G-10 currencies, from the standpoint of a Swiss investor.

In [ ]:
carry_fx_xs_returns = plot.compute_carry_fx_xs_returns(df2, currencies_list=currencies_list)
plot.plot_performances(carry_fx_xs_returns)

### Performance indicators

For each of the considered strategies, one might compute historical performance measures such as the Sharpe ratio, the average annualized return and the average and maximum drawdowns through the following code. It's also possible to make drawdown plots, histograms of returns and autocorrelation bar plots.

In [ ]:
plot.plot_return_histograms(returns, currencies_list=currencies_list)
plot.plot_return_histograms(carry_fx_xs_returns, currencies_list=currencies_list)

plot.plot_autocorrelations(carry_fx_xs_returns, currencies_list=currencies_list)

### EU risk factor: DE-IT 10y Spread

The following is an indicator to be used in a robustness check of our results; it's a measure of the European risk.

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
df['IT_minus_DE'].plot(xlabel='Time',
                       ylabel='pp',
                       title='IT-DE 10-year bond spread')
plt.savefig('latex/figures/it_de_spread.jpg')

## LaTeX

Now, let's go through the compiling process of the LaTeX file into a pdf: